In [ ]:
!pip install --upgrade pip
!pip install --upgrade setuptools
!pip install --upgrade "mxnet<2.0.0"
!pip install autogluon

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 33.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.5 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━

In [ ]:
# AutoGluonをインポートする。
from autogluon.tabular import TabularDataset, TabularPredictor, FeatureMetadata

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import scipy.stats
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
### autogluon training

In [ ]:
data_raw_tab = pd.read_csv('drive/My Drive/ML5G-PS-005_train.csv')

In [ ]:
cycleLable_train_arr = []
for i in range(0,600):
  dt1_train = data_raw_tab['label'][70*i:70*(i+1)]
  if dt1_train[70*i] != 'normal':
    cycleLable_train_arr.append(i)

In [ ]:
#read
data_train_12_mean_failure_60_noTimeLable_tab = pd.read_csv('drive/My Drive/4SA/data_train_12_failure_60_noTimeLable_tab_0to50_1025.csv')

In [ ]:
del data_train_12_mean_failure_60_noTimeLable_tab['Unnamed: 0']

In [ ]:
data_train_12_mean_failure_60_noTimeLable_tab

,index,amf.amf.app.cadvisor.container_cpu_cfs_periods,amf.amf.app.cadvisor.container_cpu_cfs_throttled_periods,amf.amf.app.cadvisor.container_cpu_cfs_throttled_seconds,amf.amf.app.cadvisor.container_cpu_system_seconds,amf.amf.app.cadvisor.container_cpu_usage_seconds,amf.amf.app.cadvisor.container_cpu_user_seconds,amf.amf.app.cadvisor.container_last_seen,amf.amf.app.cadvisor.container_memory_cache,amf.amf.app.cadvisor.container_memory_failcnt,...,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.max,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.med,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.min,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.count,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.avg,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.max,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.med,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.min,upf.upf3.infra.tcptracer._192.168.13.70_connect.count,fail60
0,0,7.018374,0,0,0.023657,0.000310,0.040218,1646023925,23730958336,0,...,0,0,0,0,0,0,0,0,0,2
1,10,8.617313,0,0,0.030356,0.002437,0.045045,1646024024,23739338752,0,...,0,0,0,0,0,0,0,0,0,2
2,20,8.655715,0,0,0.045045,0.002922,0.066243,1646024123,23748800512,0,...,0,0,0,0,0,0,0,0,0,2
3,30,8.140671,0,0,0.035819,0.005238,0.048301,1646024223,23758127104,0,...,0,0,0,0,0,0,0,0,0,2
4,40,6.803392,0,0,0.031552,0.002177,0.032538,1646024323,23767994368,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,41940,8.447730,0,0,0.033791,0.002817,0.044351,1647233323,24573620224,0,...,0,0,0,0,0,0,0,0,0,3
3596,41950,8.147513,0,0,0.034305,0.002209,0.040738,1647233423,24583217152,0,...,0,0,0,0,0,0,0,0,0,3
3597,41960,8.065487,0,0,0.042133,0.001762,0.049356,1647233523,24593219584,0,...,0,0,0,0,0,0,0,0,0,3
3598,41970,8.172922,0,0,0.034412,0.002739,0.037638,1647233623,24601735168,0,...,0,0,0,0,0,0,0,0,0,3


In [ ]:
# predictor のロード ディレクトリ名を指定する
predictor = TabularPredictor.load("drive/My Drive/4SA/model/autogluon_1025_train_norm0-50_loss10-15_failure_60_reg_cpu_4h_good",require_version_match=False)
#data_test_tab = data_train_12_mean_failure_60_noTimeLable_tab

In [ ]:
### feature importance ###

In [ ]:
data_test_tab = data_train_12_mean_failure_60_noTimeLable_tab

In [ ]:
from re import X
### data_test_tab - normal, failure index
train_failurecycle_index_arr = []
train_normalcycle_index_arr = []

x = 0
for i_cycle in range(600):
  if i_cycle in cycleLable_train_arr:
    #train_failurecycle_index_arr += [i for i in range(x, x+6)]
    train_failurecycle_index_arr.append(x)
  else:
    #train_normalcycle_index_arr += [i for i in range(x, x+6)]
    train_normalcycle_index_arr.append(x)
  x += 6

In [ ]:
train_normalcycle_index_arr

In [ ]:
data_test_tab

,index,amf.amf.app.cadvisor.container_cpu_cfs_periods,amf.amf.app.cadvisor.container_cpu_cfs_throttled_periods,amf.amf.app.cadvisor.container_cpu_cfs_throttled_seconds,amf.amf.app.cadvisor.container_cpu_system_seconds,amf.amf.app.cadvisor.container_cpu_usage_seconds,amf.amf.app.cadvisor.container_cpu_user_seconds,amf.amf.app.cadvisor.container_last_seen,amf.amf.app.cadvisor.container_memory_cache,amf.amf.app.cadvisor.container_memory_failcnt,...,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.max,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.med,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.min,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.count,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.avg,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.max,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.med,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.min,upf.upf3.infra.tcptracer._192.168.13.70_connect.count,fail60
0,0,7.018374,0,0,0.023657,0.000310,0.040218,1646023925,23730958336,0,...,0,0,0,0,0,0,0,0,0,2
1,10,8.617313,0,0,0.030356,0.002437,0.045045,1646024024,23739338752,0,...,0,0,0,0,0,0,0,0,0,2
2,20,8.655715,0,0,0.045045,0.002922,0.066243,1646024123,23748800512,0,...,0,0,0,0,0,0,0,0,0,2
3,30,8.140671,0,0,0.035819,0.005238,0.048301,1646024223,23758127104,0,...,0,0,0,0,0,0,0,0,0,2
4,40,6.803392,0,0,0.031552,0.002177,0.032538,1646024323,23767994368,0,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3595,41940,8.447730,0,0,0.033791,0.002817,0.044351,1647233323,24573620224,0,...,0,0,0,0,0,0,0,0,0,3
3596,41950,8.147513,0,0,0.034305,0.002209,0.040738,1647233423,24583217152,0,...,0,0,0,0,0,0,0,0,0,3
3597,41960,8.065487,0,0,0.042133,0.001762,0.049356,1647233523,24593219584,0,...,0,0,0,0,0,0,0,0,0,3
3598,41970,8.172922,0,0,0.034412,0.002739,0.037638,1647233623,24601735168,0,...,0,0,0,0,0,0,0,0,0,3


In [ ]:
import random

data_train_failure_random_arr = random.sample(train_failurecycle_index_arr, 20) #120 rows
data_train_normal_random_arr = random.sample(train_normalcycle_index_arr, 60) #360 rows

data_train_failure_random_tab = pd.DataFrame()
data_train_normal_random_tab = pd.DataFrame()

for i_index in range(20):
  x = data_train_failure_random_arr[i_index]
  data_train_failure_random_tab = pd.concat([data_train_failure_random_tab, data_test_tab.iloc[x:x+6, :]])

for i_index in range(60):
  x = data_train_normal_random_arr[i_index]
  data_train_normal_random_tab = pd.concat([data_train_normal_random_tab, data_test_tab.iloc[x:x+6, :]])

In [ ]:
data_train_normal_and_failure_random_tab = pd.DataFrame()
data_train_normal_and_failure_random_tab = pd.concat([data_train_normal_and_failure_random_tab, data_train_normal_random_tab])
data_train_normal_and_failure_random_tab = pd.concat([data_train_normal_and_failure_random_tab, data_train_failure_random_tab])

In [ ]:
data_train_normal_and_failure_random_tab

,index,amf.amf.app.cadvisor.container_cpu_cfs_periods,amf.amf.app.cadvisor.container_cpu_cfs_throttled_periods,amf.amf.app.cadvisor.container_cpu_cfs_throttled_seconds,amf.amf.app.cadvisor.container_cpu_system_seconds,amf.amf.app.cadvisor.container_cpu_usage_seconds,amf.amf.app.cadvisor.container_cpu_user_seconds,amf.amf.app.cadvisor.container_last_seen,amf.amf.app.cadvisor.container_memory_cache,amf.amf.app.cadvisor.container_memory_failcnt,...,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.max,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.med,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_LAST_ACK_CLOSE.usec.min,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.count,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.avg,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.max,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.med,upf.upf3.infra.tcpstates._::ffff:10.244.0.1_SYN_RECV_ESTABLISHED.usec.min,upf.upf3.infra.tcptracer._192.168.13.70_connect.count,fail60
2520,29400,8.433200,0,0,0.026631,0.002595,0.038837,1646834683,23421480960,0,...,0,0,0,0,0,0,0,0,0,3
2521,29410,8.692503,0,0,0.032597,0.001371,0.048895,1646834784,23430131712,0,...,0,0,0,0,0,0,0,0,0,3
2522,29420,7.655812,0,0,0.032349,0.003418,0.045288,1646834883,23439323136,0,...,0,0,0,0,0,0,0,0,0,3
2523,29430,8.112767,0,0,0.034479,0.001945,0.044620,1646834983,23448649728,0,...,0,0,0,0,0,0,0,0,0,3
2524,29440,7.804515,0,0,0.031590,0.002706,0.043668,1646835083,23458652160,0,...,0,0,0,0,0,0,0,0,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2101,24511,8.203281,0,0,0.041016,0.000944,0.047019,1646714433,27055316992,0,...,0,0,0,0,0,0,0,0,0,43
2102,24512,8.046902,0,0,0.031038,0.000288,0.044833,1646714443,27055857664,0,...,0,0,0,0,0,0,0,0,0,43
2103,24513,8.876317,0,0,0.031213,0.000005,0.040968,1646714453,27057074176,0,...,0,0,0,0,0,0,0,0,0,43
2104,24514,8.809321,0,0,0.027470,0.001439,0.054940,1646714463,27058290688,0,...,0,0,0,0,0,0,0,0,0,43


In [ ]:
### cheack feature importance 
# predictor.feature_importance(data_input_tab, num_shuffle_sets=20)
feature_importance=predictor.feature_importance(data_train_normal_and_failure_random_tab.iloc[:, :])
feature_importance.to_csv('drive/My Drive/4SA/featureImportance_1026_train_1to12_failure_60_reg_cpu_4h_480_goodquality.csv')

In [ ]:
feature_importance.head(30)

,importance,stddev,p_value,n,p99_high,p99_low
udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.hist.bins._64_127.count,1.122960,0.115805,1.338116e-05,5,1.361403,0.884516
smf.smf.app.cadvisor.container_memory_rss,1.105357,0.099938,7.931820e-06,5,1.311130,0.899583
amf.amf.app.cadvisor.container_memory_rss,0.897030,0.143522,7.602252e-05,5,1.192543,0.601517
ausf.ausf.app.cadvisor.container_memory_rss,0.778589,0.103861,3.711220e-05,5,0.992440,0.564739
amf.amf.infra.tcpwin._192.168.13.80_192.168.13.82.snd_cwnd.hist.bins._4_7.count,0.547578,0.090407,8.601593e-05,5,0.733726,0.361429
udm.udm.app.cadvisor.container_memory_rss,0.530349,0.074055,4.445719e-05,5,0.682829,0.377869
upf.upf3.app.cadvisor.container_memory_rss,0.451554,0.110517,3.982442e-04,5,0.679111,0.223997
udm.udm.infra.tcprtt._192.168.13.70_192.168.13.80.stat.avg,0.439769,0.067508,6.459376e-05,5,0.578770,0.300769
amf.amf.infra.tcpwin._192.168.13.80_192.168.13.82.snd_cwnd.hist.stat.min,0.376355,0.054657,5.191246e-05,5,0.488896,0.263815
udm.udm.infra.tcprtt._192.168.13.70_192.168.13.82.stat.max,0.263546,0.066296,4.425007e-04,5,0.400049,0.127042
